<a href="https://colab.research.google.com/github/yeesem/Advanced-Tensorflow-Specialization/blob/main/Neural_Style_Transfer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Imports

In [ ]:
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
from keras import backend as K

from imageio import mimsave
from IPython.display import display as display_fn
from IPython.display import Image, clear_output

# Utiities

In [ ]:
def tensor_to_image(tensor):
  # tf.shape(tensor) returns the shape of the tensor as a TensorFlow tensor.
  # For example, if the input tensor has a shape of (1, 64, 64, 3),
  # tensor_shape will store this shape.
  tensor_shape = tf.shape(tensor)
  # tf.shape(tensor_shape) returns the shape of tensor_shape. Since tensor_shape
  # itself is a tensor, tf.shape(tensor_shape) returns its shape, which will be a
  # single number indicating the number of dimensions in tensor_shape. Typically,
  # number_elem_shape will be [4] for a 4D tensor shape.
  number_elem_shape = tf.shape(tensor_shape)
  # This line checks if the number of dimensions in tensor is greater than 3.
  # For most image data tensors used in deep learning, the shape would be either
  # 3D (height, width, channels) or 4D (batch, height, width, channels).
  if number_elem_shape > 3:
    assert tensor_shape[0] == 1
    # If the tensor has more than 3 dimensions, it assumes that the first dimension
    # is the batch size and checks if it is 1 (i.e., a single image). If this is true,
    # it removes the batch dimension by setting tensor to tensor[0].
    tensor = tensor[0]
  return tf.keras.preprocessing.image.array_to_img(tensor)

def load_img(path_to_img):
  # max_dim is set to 512, which will be the maximum dimension (width or height) of
  # the image after resizing.
  max_dim = 512
  # reads the image file from the given path.
  image = tf.io.read_file(path_to_img)
  # decodes the JPEG-encoded image to a tensor.
  image = tf.image.decode_jpeg(image)
  # This line converts the image tensor to tf.float32 data type, with pixel values
  # scaled to the range [0, 1].
  image = tf.image.convert_image_dtype(image,tf.float32)

   # tf.shape(image)[:-1] gets the shape of the image excluding the last dimension
   # (which represents the color channels). This results in a tensor representing the
   # height and width of the image. tf.cast(tf.shape(image)[:-1], tf.float32) casts
   # this shape tensor to tf.float32.
  shape = tf.shape(image)[:-1]
  shape = tf.cast(tf.shape(image)[:-1],tf.float32)
  long_dim = max(shape)
  scale = max_dim / long_dim

  new_shape = tf.cast(shape * scale, tf.int32)

  image = tf.image.resize(image,new_shape)
  image = image[tf.newaxis,:]
  image = tf.image.convert_image_dtype(image,tf.uint8)

  return image

# Download Images

In [ ]:
IMAGE_DIR = 'images'

# create directory
!mkdir {IMAGE_DIR}

# download images to the directory you just created
!wget -q -O ./images/cafe.jpg https://cdn.pixabay.com/photo/2018/07/14/15/27/cafe-3537801_1280.jpg
!wget -q -O ./images/swan.jpg https://cdn.pixabay.com/photo/2017/02/28/23/00/swan-2107052_1280.jpg
!wget -q -O ./images/tnj.jpg https://i.dawn.com/large/2019/10/5db6a03a4c7e3.jpg
!wget -q -O ./images/rudolph.jpg https://cdn.pixabay.com/photo/2015/09/22/12/21/rudolph-951494_1280.jpg
!wget -q -O ./images/dynamite.jpg https://cdn.pixabay.com/photo/2015/10/13/02/59/animals-985500_1280.jpg
!wget -q -O ./images/painting.jpg https://storage.googleapis.com/download.tensorflow.org/example_images/Vassily_Kandinsky%2C_1913_-_Composition_7.jpg

print("image files you can choose from: ")
!ls images

image files you can choose from: 
cafe.jpg  dynamite.jpg	painting.jpg  rudolph.jpg  swan.jpg  tnj.jpg
